In [51]:
import logging
import os
import boto3 # Need to install
from botocore.exceptions import ClientError
import smart_open # Need to install
import tensorflow as tf # Need to install
import sys
import os
import pandas as pd
import nltk
#from tensorflow.python.lib.io import file_io

In [2]:
def list_bucket_objects(bucket_name):
    """List the objects in an Amazon S3 bucket

    :param bucket_name: string
    :return: List of bucket objects. If error, return None.
    """

    # Retrieve the list of bucket objects
    s_3 = boto3.client('s3')
    try:
        response = s_3.list_objects_v2(Bucket=bucket_name)
    except ClientError as e:
        # AllAccessDisabled error == bucket not found
        logging.error(e)
        return None

    # Only return the contents if we found some keys
    if response['KeyCount'] > 0:
        return response['Contents']

    return None

In [3]:
bucket_name = 'amazon-reviews-pds'
list_bucket_objects(bucket_name)

[{'Key': 'LICENSE.txt',
  'LastModified': datetime.datetime(2017, 11, 28, 15, 9, 41, tzinfo=tzlocal()),
  'ETag': '"042608329ee8bb72ff55c152ef88db5e"',
  'Size': 1407,
  'StorageClass': 'STANDARD'},
 {'Key': 'license.txt',
  'LastModified': datetime.datetime(2018, 4, 11, 18, 39, 23, tzinfo=tzlocal()),
  'ETag': '"056f282f80567222fffeef309eb5a2f4"',
  'Size': 1389,
  'StorageClass': 'STANDARD'},
 {'Key': 'parquet/product_category=Apparel/part-00000-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet',
  'LastModified': datetime.datetime(2018, 4, 9, 6, 35, 35, tzinfo=tzlocal()),
  'ETag': '"ee9c7278fd88759213dd02b7147d3eba-15"',
  'Size': 120549202,
  'StorageClass': 'STANDARD'},
 {'Key': 'parquet/product_category=Apparel/part-00001-495c48e6-96d6-4650-aa65-3c36a3516ddd.c000.snappy.parquet',
  'LastModified': datetime.datetime(2018, 4, 9, 6, 35, 35, tzinfo=tzlocal()),
  'ETag': '"dad7ad6f78becb84c19d51a3cea781f7-15"',
  'Size': 120859922,
  'StorageClass': 'STANDARD'},
 {'Key': 'parq

In [5]:
bucket_name = 'amazon-reviews-pds'
object_key = 'tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz'
path = 's3://{}/{}'.format(bucket_name, object_key)
electronics_df = pd.read_table(smart_open.open(path), error_bad_lines=False, sep='\t', quoting=3)

# From what year are the reviews from

In [ ]:
electronics_df['review_date'] = electronics_df['review_date'].astype('datetime64')
electronics_df['review_year'] = electronics_df['review_date'].dt.year

In [79]:
electronics_df['review_year'].value_counts()

2014    833159
2015    811696
2013    556228
2012    268650
2011    187125
2010    123491
2009     91080
2008     71183
2007     61353
2006     29576
2005     20098
2004     12601
2003      9828
2002      7176
2001      5481
2000      4443
1999       701
Name: review_year, dtype: int64

# How many different products are there?

In [21]:
electronics_df['product_parent'].nunique()

166244

# How many reviews are there for electronics?

In [23]:
electronics_df.shape[0]

3093869

In [36]:
electronics_df.groupby('review_date')['review_id'].count()

review_date
1999-06-09       1
1999-06-12       2
1999-06-13       1
1999-06-14       2
1999-06-17       1
              ... 
2015-08-27    3654
2015-08-28    2809
2015-08-29    2999
2015-08-30    2873
2015-08-31    3459
Name: review_id, Length: 5904, dtype: int64

# How many reviews per product are there?

In [28]:
electronics_df.groupby('product_parent')['review_id'].count().describe()e

count    166244.000000
mean         18.610410
std         158.218827
min           1.000000
25%           1.000000
50%           2.000000
75%           6.000000
max       24828.000000
Name: review_id, dtype: float64

In [34]:
electronics_df.groupby(['product_parent', 'product_title'])['review_id']\
              .count()\
              .to_frame('review_count')\
              .sort_values(by='review_count', ascending=False)\
              .head(10)

,,review_count
product_parent,product_title,
921315865,Panasonic ErgoFit In-Ear Earbud Headphone,24813
443415697,"Mediabridge ULTRA Series HDMI Cable (3 Foot) - High-Speed Supports Ethernet, 3D and Audio Return [Newest Standard]",15675
617978254,"AmazonBasics High-Speed HDMI Cable - 6.5 Feet (2 Meters) Supports Ethernet, 3D, 4K and Audio Return",15348
808084969,Clip Plus 4 GB MP3 Player (Black),11779
871765831,"High Speed HDMI Cable (1.5 Feet) With Ethernet - CL3 Certified - Supports 3D and Audio Return Channel, 1-Pack",11177
572574607,AmazonBasics High Speed HDMI Cable,10740
851902359,"VideoSecu ML531BE TV Wall Mount for most 22""-55"" LED LCD Plasma Flat Screen Monitor up to 88 lb VESA 400x400 with Full Motion Swivel Articulating 20 in Extension Arm, HDMI Cable & Bubble Level WP5",10216
862565728,CABTE High speed HDMI 1.4 HDMI cable 10ft 1080p with mesh&filters supports 3D&blue ray,9881
845709830,"Cheetah APTMM2B TV Wall Mount for 20-75"" TVs up to VESA 600 and 165lbs, and fits 16” And 24” Wall Studs, and includes a Tilt TV Bracket, a 10' Twisted Veins HDMI Cable and a 6"" 3-Axis Magnetic Bubble Level",9365


# How many vine reviews are there?

In [42]:
electronics_df[electronics_df['vine'] == 'Y'].shape[0]

18512

# How long are the reviews on average

In [58]:
electronics_df['review_body'] = electronics_df['review_body'].astype(str)
electronics_df['review_len'] = electronics_df['review_body'].apply(lambda x: len(x.split()))

In [65]:
electronics_df['review_len'].describe().round(2)

count    3093869.00
mean          68.94
std          115.43
min            1.00
25%           20.00
50%           35.00
75%           78.00
max         8360.00
Name: review_len, dtype: float64

In [68]:
electronics_df[electronics_df['product_title'].str.contains('HDTV')]

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,samsung_ind,review_len
36,US,15280140,R2WTXV596WTY43,B00RFLXB9Y,251695655,1byone OUS00-0565 Shiny Antenna Super Thin HDT...,Electronics,5,0,0,N,Y,Five Stars,"Works great in Charlotte, NC. Received all the...",2015-08-31,False,15
68,US,15549681,R1ZID06Z5V2TNO,B00DHKKI16,789810342,Mohu Leaf 50 Indoor HDTV Antenna (Certified Re...,Electronics,5,0,0,N,Y,Save some money,My son gets 41 channels in Hickory NC. He is w...,2015-08-31,False,12
95,US,12681886,R2ZGM1SYG0K2OO,B00RFLGJLG,833935017,1byone OUS00-0562 Amplified HDTV Antenna 50 Mi...,Electronics,4,0,0,N,Y,Worked well the Caribbean,I purchased the OUS00-0563 Super Thin Indoor H...,2015-08-31,False,39
118,US,14658280,R14C2RHHZB6K2Y,B00IF70TSI,677825593,"1byone Amplified HDTV Antenna, with Detachable...",Electronics,2,3,3,N,Y,Two Stars,Too expensive & reception is not great. A $10 ...,2015-08-31,False,14
149,US,22094303,R3DDO8K9OK23JU,B001GGAIIQ,599577698,"RCA Multi-Directional, Flat Digital Indoor HDT...",Electronics,5,1,1,N,Y,Installed and I was able to pull up all the ...,Installed and I was able to pull up all the pu...,2015-08-31,False,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3081501,US,48950603,R2URQI7DUQT1RH,B00005T3A9,194604442,Terk Technology TV-55 UHF/VHF HDTV Indoor/Outd...,Electronics,1,93,104,N,N,A poor performer...,"I bought Terk TV55 for my RCA F38310, an HDTV....",2002-03-23,False,134
3081635,US,45598501,RU9NFUL9BCWTP,B00005T3A9,194604442,Terk Technology TV-55 UHF/VHF HDTV Indoor/Outd...,Electronics,2,40,41,N,N,"OK, but rabbit ears are better",Purchased this unit (mounted indoors) because ...,2002-03-15,False,43
3084275,US,47065303,R2SCNU3XWZAENV,B00003CWAY,413891340,Monster Cable J2 DCAM4/4-6 IEEE-1394 4-pin Dig...,Electronics,1,21,22,N,N,Another overpriced digital cable!,Let me start by stating that I have nothing pe...,2001-11-19,False,151
3091465,US,50128237,R2ZTKYGWXZAHCF,B00003CWAY,413891340,Monster Cable J2 DCAM4/4-6 IEEE-1394 4-pin Dig...,Electronics,5,3,12,N,N,THE BEST! BUY IT NOW!,"made digital remastering easy as ever, i use i...",2000-06-17,False,15
